# 解剖学・骨学実習〜計測・解析編2〜

[伊藤　毅](https://github.com/itots)  
2021-12-17

このドキュメントは、解剖学・骨学実習の計測・解析編第２部の補足資料です。第2部では、モルフォロジー処理、セグメンテーション、レジストレーションなどの画像処理を扱います。演習を通して、それらの初歩的な概念を理解することを目的とします。（第1部の補足資料は[こちら](https://itots.github.io/morphometrics_lecture/index.html)です。）

## ソフトウェアの準備
### Anaconda　　
計算科学のためのPythonの総合開発環境です。  
以下から対応するOSのインストーラをダウンロードし、指示に従ってインストールしてください。  
https://www.anaconda.com/products/individual

### SimpleITK
画像処理のライブラリです。  
Anacondaのインストール後、ターミナル（Windowsの場合は、スタートメニュー/Anaconda 3/Anaconda Prompt）を開いて、以下を入力・実行し、指示に従ってインストールしてください。
```
conda install -c simpleitk simpleitk
```
### elastix
レジストレーションのためのソフトウェアです。  
以下から対応するOSの圧縮フォルダをダウンロードし、適当な場所に保存して展開してください。  
https://github.com/SuperElastix/elastix/releases/tag/5.0.1　   

なお、今回は環境構築の簡略化のためコンパイル済のelastixソフトウェアを使用しますが、Pythonのパッケージ（[SimpleElastix](https://simpleelastix.github.io/)）としての利用も可能です。


#### Macの場合
`~/.zshrc`（もしくは`~/.bash_profile`）に以下を追記してください。`folder`は、先ほどダウンロード・展開したelastixのフォルダのパスです。
```Zsh
export PATH=folder/bin:$PATH
export DYLD_LIBRARY_PATH=folder/lib:$DYLD_LIBRARY_PATH
```
#### Windowsの場合
設定の「環境変数を編集」で、先ほどダウンロード・展開したelastixのフォルダのパスを追加してください。  
具体的には、ユーザーの環境変数のPathを選択→編集→新規→パスを入力→OKです。

### 3D Slicer
画像処理のGUIソフトウェアです。  
以下から対応するOSのインストーラをダウンロードし、指示に従ってインストールしてください。  
https://download.slicer.org/

### 確認
ターミナル（Windowsの場合はAnaconda Prompt）で、以下を入力実行し、エラーが出ないことを確認してください。
```Zsh
elastix --version
```
```Zsh
python
```
（ここで、`>>>`のようなプロンプトに変わるので、その状態で以下を入力実行してください。）
```Python
import SimpleITK
```

## 画像データの基本

### 使用するDICOMデータ
[Digital Morphology Museum, Kyoto University](http://dmm.pri.kyoto-u.ac.jp/dmm/WebGallery/index.html)の以下のDICOMデータを使用しますが、自分のデータでも試してみると面白いと思います（ただし大きすぎるデータは普通のPCでは扱えないかもしれません）。  

| PRICT ID | PRISK ID | Species | Sex |
| ---- | ---- | ---- | ---- |
| [2739](http://dmm.pri.kyoto-u.ac.jp/dmm/WebGallery/dicom/dicomProperty.html?id=2797) | 7379 | <i>Macaca fuscata</i> | female |
| [1010](http://dmm.pri.kyoto-u.ac.jp/dmm/WebGallery/dicom/dicomProperty.html?id=1014) | 9361 | <i>Macaca fuscata</i> | male |
| [1465](http://dmm.pri.kyoto-u.ac.jp/dmm/WebGallery/dicom/dicomProperty.html?id=1477) | 3027 | <i>Macaca mulatta</i> | female |  

以下の階層構造を持つとします。
```Zsh
.
├── dicom
│   ├── PRI-3027
│   │   ├── instance_2693.dcm
│   │   ├── instance_2694.dcm
│   │           （省略）
│   ├── PRI-7379
│   └── PRI-9361
├── image_analysis.ipynb
└── parameters
```

### SimpleITKでDICOMデータを読み込む
Insight Segmentation and Registration Toolkit (ITK)は、セグメンテーションとレジストレーションをはじめとする画像解析の開発のためのオープンソースでクロスプラットフォームなライブラリで、様々な画像解析ソフトウェアで使用されています。
SimpleITKは、ITKを簡略化したプログラミングインターフェイスです（Beare et al. 2018, Yaniv et al. 2018, Lowekamp et al. 2013）。以下コードは、SimpleITK Notebooks (https://github.com/InsightSoftwareConsortium/SimpleITK-Notebooks) を参考にしています。

In [ ]:
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
import subprocess
import os
import pandas as pd
from skimage.measure import profile_line

DICOMデータ（PRI-7379）を読み込みます。

In [ ]:
dicom_names = sitk.ImageSeriesReader_GetGDCMSeriesFileNames("./dicom/PRI-7379")
img = sitk.ReadImage(dicom_names)

読み込み関数を作成します。

In [ ]:
def read_dicom(path_to_dir):
    dicom_names = sitk.ImageSeriesReader_GetGDCMSeriesFileNames(path_to_dir)    
    sitk.ProcessObject_SetGlobalWarningDisplay(False) # ピクセルサイズとスライス間隔が異なると警告が出るため。
    img = sitk.ReadImage(dicom_names)
    sitk.ProcessObject_SetGlobalWarningDisplay(True)
    return img

In [ ]:
img = read_dicom("./dicom/PRI-7379/")

### 画像の特性
SimpleITK（ITK）では、画像データは次のように定義されます。　　
> image is defined by a set of points on a grid occupying a physical region in space  

画像の領域（physical region）は、Origin（原点）、Spacing（ピクセル間間隔）、Size（ピクセル数）、Direction cosine matrix（方向余弦行列）で定義されます。
> - Origin (vector like type) - location in the world coordinate system of the voxel with all zero indexes.
> - Spacing (vector like type) - distance between pixels along each of the dimensions.
> - Size (vector like type) - number of pixels in each dimension.
> - Direction cosine matrix (vector like type representing matrix in row major order) - direction of each of the axes corresponding to the matrix columns.

出典：https://simpleitk.readthedocs.io/en/master/fundamentalConcepts.html#  

先ほど読み込んだPRI-7379の属性値を確認します。

In [ ]:
print("Size:", img.GetSize())
print("Origin:", img.GetOrigin())
print("Spacing", img.GetSpacing())
print("Direction", img.GetDirection())

SimpleITKのImageが持つ属性は`help()`で確認できます。

In [ ]:
help(img)

### ImageからNumpy配列への変換
ImageからNumPyのndarray（多次元配列）に変換します。

In [ ]:
nda = sitk.GetArrayFromImage(img)
print(nda)

SimpleITKとNumPyでは軸の順番が異なることに注意が必要です。  
- SimpleITK: [x,y,z]  
- NumPy: [z,y,x]

In [ ]:
print("SimpleITK img size", img.GetSize())
print("NumPy nda size", nda.shape)

### 画像の描画
SimpleITKのImageクラスは描画メソッドを持ちません。  

In [ ]:
plt.imshow(nda[200], cmap='gray')

In [ ]:
plt.imshow(nda[:,200,:], cmap='gray')

Spacing（ピクセル間間隔）が不均一なことに注意してください。

In [ ]:
plt.imshow(nda[:,200,:], cmap='gray', aspect = img.GetSpacing()[2]/img.GetSpacing()[1])

軸を物理距離（mm）で表示します。

In [ ]:
plt.imshow(nda[:,200,:], cmap='gray', extent = (0,nda.shape[2]*img.GetSpacing()[0],nda.shape[0]*img.GetSpacing()[2],0))

描画関数を作成します。

In [ ]:
def imshow(img, 
           nonuniform=False, alpha=1, cmap='gray', mask=None, phys_axis=False,
           h_min=None, h_max=None, v_min=None, v_max=None):
    
    nda = sitk.GetArrayFromImage(img).T # imgとndaの軸の順番の違いに注意。

    if mask is None:
        nda_show = nda
    else:
        nda_show = np.ma.masked_where(nda < mask, nda)
    
    if phys_axis:
        extent = (0, nda.shape[0]*img.GetSpacing()[0], nda.shape[1]*img.GetSpacing()[1], 0)
        aspect = 1
    else:
        extent = (0, nda.shape[0], nda.shape[1], 0)
        if nonuniform:
            aspect = img.GetSpacing()[1]/img.GetSpacing()[0]
        else:
            aspect = 1
    
    plt.imshow(nda_show.T, cmap=cmap, alpha=alpha, aspect=aspect, extent=extent)
    if h_min is not None and h_max is not None: 
        plt.xlim(h_min,h_max)
    if v_min is not None and v_max is not None: 
        plt.ylim(v_min,v_max)
        
    if phys_axis:
        plt.xlabel("mm")
        plt.ylabel("mm")  

In [ ]:
imshow(img[:,:,200])

### NumPy配列からImageへの変換
Origin、Spacing、Directionなどがリセットされていることに注意してください。

In [ ]:
img_rev = sitk.GetImageFromArray(nda)

In [ ]:
print("Size:", img_rev.GetSize())
print("Origin:", img_rev.GetOrigin())
print("Spacing", img_rev.GetSpacing())
print("Direction", img_rev.GetDirection())

属性値をリストアします。

In [ ]:
img_rev.CopyInformation(img)

In [ ]:
print("Size:", img_rev.GetSize())
print("Origin:", img_rev.GetOrigin())
print("Spacing", img_rev.GetSpacing())
print("Direction", img_rev.GetDirection())

## モルフォロジー演算

Morphological operation（モルフォロジー演算）は、主に2値化画像を対象とした、形状に基づく画像処理技術のことです。  
  
グローバル閾値（$> -700$）でセグメンテーション（2値化）します。

In [ ]:
seg_th0 = img > -700

In [ ]:
plt.figure(figsize=(10,10))
imshow(seg_th0[:,:,30], v_min=250, v_max=0, h_min=125, h_max=375)

### Erosion（収縮）

In [ ]:
seg_th0_er = sitk.BinaryErode(seg_th0)
fig = plt.figure(figsize=(20, 10))
fig.add_subplot(1, 2, 1)
plt.title("original")
imshow(seg_th0[:,:,30], v_min=250, v_max=0, h_min=125, h_max=375)
fig.add_subplot(1, 2, 2)
plt.title("erosion")
imshow(seg_th0_er[:,:,30], v_min=250, v_max=0, h_min=125, h_max=375)

### Dilation（膨張）

In [ ]:
seg_th0_dl = sitk.BinaryDilate(seg_th0)
fig = plt.figure(figsize=(20, 10))
fig.add_subplot(1, 2, 1)
plt.title("original")
imshow(seg_th0[:,:,30], v_min=250, v_max=0, h_min=125, h_max=375)
fig.add_subplot(1, 2, 2)
plt.title("dilation")
imshow(seg_th0_dl[:,:,30], v_min=250, v_max=0, h_min=125, h_max=375)

### ErosionとDilationの組み合わせ
順番によって結果が異なります。

- erosion → dilation：島（ノイズ）の除去
- dilation → erosion：穴埋め

In [ ]:
seg_th0_er_dl = sitk.BinaryDilate(seg_th0_er)
seg_th0_dl_er = sitk.BinaryErode(seg_th0_dl)

fig = plt.figure(figsize=(20, 10))
fig.add_subplot(1, 3, 1)
plt.title("original")
imshow(seg_th0[:,:,30], v_min=250, v_max=0, h_min=125, h_max=375)
fig.add_subplot(1, 3, 2)
plt.title("dilation after erosion")
imshow(seg_th0_er_dl[:,:,30], v_min=250, v_max=0, h_min=125, h_max=375)
fig.add_subplot(1, 3, 3)
plt.title("erosion after dilation")
imshow(seg_th0_dl_er[:,:,30], v_min=250, v_max=0, h_min=125, h_max=375)

### OpeningとClosing
これらの処理は、それぞれopeningとclosingと呼ばれ、専用の関数が用意されています。 

In [ ]:
seg_th0_op = sitk.BinaryMorphologicalOpening(seg_th0)
seg_th0_cl = sitk.BinaryMorphologicalClosing(seg_th0)

fig = plt.figure(figsize=(20, 10))
fig.add_subplot(1, 3, 1)
plt.title("original")
imshow(seg_th0[:,:,30], v_min=250, v_max=0, h_min=125, h_max=375)
fig.add_subplot(1, 3, 2)
plt.title("opening")
imshow(seg_th0_op[:,:,30], v_min=250, v_max=0, h_min=125, h_max=375)
fig.add_subplot(1, 3, 3)
plt.title("closing")
imshow(seg_th0_cl[:,:,30], v_min=250, v_max=0, h_min=125, h_max=375)

## セグメンテーション

セグメンテーションは、画像データをいくつかのセグメントに分割する画像処理技術のことです。とくに２つのセグメントに分割することを2値化ともいいます。

### Half-maximum height (HMH)
[ここ](https://itots.github.io/morphometrics_lecture/index.html#33_3D%E3%82%B5%E3%83%BC%E3%83%95%E3%82%A7%E3%82%B9%E3%83%A2%E3%83%87%E3%83%AB%E3%81%AE%E4%BD%9C%E6%88%90)も参考にしてください。

In [ ]:
profile0 = profile_line(nda[60],[220,190],[205,205], linewidth=1, mode='reflect')

fig = plt.figure(figsize=(20, 10))
fig.add_subplot(1, 2, 1)
plt.plot([190,205], [220,205], color='orange')
imshow(img[:,:,60], v_max=150, v_min=250, h_min=150, h_max=250)
fig.add_subplot(1, 2, 2)
plt.plot(profile0, color='orange')

In [ ]:
hmh0 = (profile0.max() + profile0.min())/2
print("HMH: ", hmh0)

In [ ]:
profile1 = profile_line(nda[250],[425,145], [410,160], linewidth=1, mode='reflect')

fig = plt.figure(figsize=(20, 10))
fig.add_subplot(1, 2, 1)
plt.plot([145,160], [425,410], color='orange')
imshow(img[:,:,250], v_max=380, v_min=450, h_min=115, h_max=185)
fig.add_subplot(1, 2, 2)
plt.plot(profile1, color='orange')

In [ ]:
hmh1 = (profile1.max() + profile1.min())/2
print("HMH: ", hmh1)

### グローバル閾値

In [ ]:
seg_th1 = img > hmh0
seg_th2 = img > hmh1

In [ ]:
fig = plt.figure(figsize=(20, 10))
fig.add_subplot(1, 4, 1)
imshow(img[:,:,60], v_max=150, v_min=250, h_min=150, h_max=250)
fig.add_subplot(1, 4, 2)
plt.title(hmh0)
imshow(img[:,:,60], v_max=150, v_min=250, h_min=150, h_max=250)
imshow(seg_th1[:,:,60], v_max=150, v_min=250, h_min=150, h_max=250, alpha=0.5, cmap='Set2', mask=1)
fig.add_subplot(1, 4, 3)
imshow(img[:,:,60], v_max=150, v_min=250, h_min=150, h_max=250)
imshow(seg_th1[:,:,60], v_max=150, v_min=250, h_min=150, h_max=250, alpha=0.5, cmap='Set2', mask=1)
imshow(seg_th2[:,:,60], v_max=150, v_min=250, h_min=150, h_max=250, alpha=0.5, cmap='Set1', mask=1)
fig.add_subplot(1, 4, 4)
plt.title(hmh1)
imshow(img[:,:,60], v_max=150, v_min=250, h_min=150, h_max=250)
imshow(seg_th2[:,:,60], v_max=150, v_min=250, h_min=150, h_max=250, alpha=0.5, cmap='Set1', mask=1)

In [ ]:
fig = plt.figure(figsize=(20, 10))
fig.add_subplot(1, 4, 1)
imshow(img[:,:,250], v_max=380, v_min=450, h_min=115, h_max=185)
fig.add_subplot(1, 4, 2)
plt.title(hmh0)
imshow(img[:,:,250], v_max=380, v_min=450, h_min=115, h_max=185)
imshow(seg_th1[:,:,250], v_max=380, v_min=450, h_min=115, h_max=185, alpha=0.5, cmap='Set2', mask=1)
fig.add_subplot(1, 4, 3)
imshow(img[:,:,250], v_max=380, v_min=450, h_min=115, h_max=185)
imshow(seg_th1[:,:,250], v_max=380, v_min=450, h_min=115, h_max=185, alpha=0.5, cmap='Set2', mask=1)
imshow(seg_th2[:,:,250], v_max=380, v_min=450, h_min=115, h_max=185, alpha=0.5, cmap='Set1', mask=1)
fig.add_subplot(1, 4, 4)
plt.title(hmh1)
imshow(img[:,:,250], v_max=380, v_min=450, h_min=115, h_max=185)
imshow(seg_th2[:,:,250], v_max=380, v_min=450, h_min=115, h_max=185, alpha=0.5, cmap='Set1', mask=1)

### エッジ検出

- Prewitt
- Sobel
- Canny  

などの方法があります。
今回はSobelフィルタを使用します。  

Sobel処理は、通常$3\times3$のカーネル（フィルタ）を用いて、畳み込みを行います。
![sobel operation](figure/sobel.png)  
Sobel処理の概念図

In [ ]:
sobel = sitk.SobelEdgeDetection(sitk.Cast(sitk.RescaleIntensity(img), sitk.sitkFloat64))
plt.figure(figsize=(15,15))
imshow(sobel[:,:,200])

-700以下をマスクします。

In [ ]:
img_msk = sitk.Mask(img, img > -700, -1000, 0)
sobel = sitk.SobelEdgeDetection(sitk.Cast(sitk.RescaleIntensity(img_msk), sitk.sitkFloat64))
plt.figure(figsize=(15,15))
imshow(sobel[:,:,200])

### 分水嶺法
領域指定のための種（seeds）を準備します。

In [ ]:
bone = sitk.BinaryErode(img > -500)
air = img < -1000
bone_air = bone + air * 2
fig = plt.figure(figsize=(15,10))
fig.add_subplot(1, 2, 1)
plt.title("bone seed")
imshow(sobel[:,:,200])
imshow(bone[:,:,200], cmap = "Wistia", alpha = 0.3, mask=1)
fig.add_subplot(1, 2, 2)
plt.title("air seed")
imshow(sobel[:,:,200])
imshow(air[:,:,200], cmap = "cool", alpha = 0.3, mask=1)

勾配の大きさの分水嶺を境界としてセグメンテーションします。

In [ ]:
seg_ws = sitk.MorphologicalWatershedFromMarkers(sobel, bone_air, markWatershedLine=False, fullyConnected=False)
seg_ws = seg_ws < 2  # 空気のラベルの除去

In [ ]:
plt.figure(figsize=(15,15))
imshow(img[:,:,200])
imshow(seg_ws[:,:,200], cmap = "Wistia", alpha = 0.5, mask=1)

In [ ]:
fig = plt.figure(figsize=(20, 10))
fig.add_subplot(2, 4, 1)
imshow(img[:,:,60], v_max=150, v_min=250, h_min=150, h_max=250)
fig.add_subplot(2, 4, 2)
plt.title(hmh0)
imshow(img[:,:,60], v_max=150, v_min=250, h_min=150, h_max=250)
imshow(seg_th1[:,:,60], v_max=150, v_min=250, h_min=150, h_max=250, alpha=0.5, cmap='Set2', mask=1)
fig.add_subplot(2, 4, 3)
imshow(img[:,:,60], v_max=150, v_min=250, h_min=150, h_max=250)
imshow(seg_th1[:,:,60], v_max=150, v_min=250, h_min=150, h_max=250, alpha=0.5, cmap='Set2', mask=1)
imshow(seg_ws[:,:,60], v_max=150, v_min=250, h_min=150, h_max=250, alpha=0.5, cmap='Wistia', mask=1)
fig.add_subplot(2, 4, 4)
plt.title("watershed")
imshow(img[:,:,60], v_max=150, v_min=250, h_min=150, h_max=250)
imshow(seg_ws[:,:,60], v_max=150, v_min=250, h_min=150, h_max=250, alpha=0.5, cmap='Wistia', mask=1)

fig.add_subplot(2, 4, 5)
imshow(img[:,:,60], v_max=150, v_min=250, h_min=150, h_max=250)
fig.add_subplot(2, 4, 6)
plt.title(hmh1)
imshow(img[:,:,60], v_max=150, v_min=250, h_min=150, h_max=250)
imshow(seg_th2[:,:,60], v_max=150, v_min=250, h_min=150, h_max=250, alpha=0.5, cmap='Set1', mask=1)
fig.add_subplot(2, 4, 7)
imshow(img[:,:,60], v_max=150, v_min=250, h_min=150, h_max=250)
imshow(seg_ws[:,:,60], v_max=150, v_min=250, h_min=150, h_max=250, alpha=0.5, cmap='Wistia', mask=1)
imshow(seg_th2[:,:,60], v_max=150, v_min=250, h_min=150, h_max=250, alpha=0.5, cmap='Set1', mask=1)
fig.add_subplot(2, 4, 8)
plt.title("watershed")
imshow(img[:,:,60], v_max=150, v_min=250, h_min=150, h_max=250)
imshow(seg_ws[:,:,60], v_max=150, v_min=250, h_min=150, h_max=250, alpha=0.5, cmap='Wistia', mask=1)

In [ ]:
fig = plt.figure(figsize=(20, 10))
fig.add_subplot(2, 4, 1)
imshow(img[:,:,250], v_max=380, v_min=450, h_min=115, h_max=185)
fig.add_subplot(2, 4, 2)
plt.title(hmh0)
imshow(img[:,:,250], v_max=380, v_min=450, h_min=115, h_max=185)
imshow(seg_th1[:,:,250], v_max=380, v_min=450, h_min=115, h_max=185, alpha=0.5, cmap='Set2', mask=1)
fig.add_subplot(2, 4, 3)
imshow(img[:,:,250], v_max=380, v_min=450, h_min=115, h_max=185)
imshow(seg_th1[:,:,250], v_max=380, v_min=450, h_min=115, h_max=185, alpha=0.5, cmap='Set2', mask=1)
imshow(seg_ws[:,:,250], v_max=380, v_min=450, h_min=115, h_max=185, alpha=0.5, cmap='Wistia', mask=1)
fig.add_subplot(2, 4, 4)
plt.title("watershed")
imshow(img[:,:,250], v_max=380, v_min=450, h_min=115, h_max=185)
imshow(seg_ws[:,:,250], v_max=380, v_min=450, h_min=115, h_max=185, alpha=0.5, cmap='Wistia', mask=1)

fig.add_subplot(2, 4, 5)
imshow(img[:,:,250], v_max=380, v_min=450, h_min=115, h_max=185)
fig.add_subplot(2, 4, 6)
plt.title(hmh1)
imshow(img[:,:,250], v_max=380, v_min=450, h_min=115, h_max=185)
imshow(seg_th2[:,:,250], v_max=380, v_min=450, h_min=115, h_max=185, alpha=0.5, cmap='Set1', mask=1)
fig.add_subplot(2, 4, 7)
imshow(img[:,:,250], v_max=380, v_min=450, h_min=115, h_max=185)
imshow(seg_ws[:,:,250], v_max=380, v_min=450, h_min=115, h_max=185, alpha=0.5, cmap='Wistia', mask=1)
imshow(seg_th2[:,:,250], v_max=380, v_min=450, h_min=115, h_max=185, alpha=0.5, cmap='Set1', mask=1)
fig.add_subplot(2, 4, 8)
plt.title("watershed")
imshow(img[:,:,250], v_max=380, v_min=450, h_min=115, h_max=185)
imshow(seg_ws[:,:,250], v_max=380, v_min=450, h_min=115, h_max=185, alpha=0.5, cmap='Wistia', mask=1)

### 結果の確認

セグメンテーションの結果を出力し、3D Slicer等で確認してください。  
なお、Visualization Toolkit（VTK）などを利用することで、Pythonでサーフェスモデルを作成することも可能です。

In [ ]:
os.makedirs('segmentation', exist_ok=True)
sitk.WriteImage(seg_ws, 'segmentation/watershged.mhd')

### その他のセグメンテーションの方法
- 深層学習：[Biomedisa](https://biomedisa.org/)など
- ローカル閾値
- レジストレーション
- 手動  
など

## レジストレーション

レジストレーション（image registration）は、画像の重ね合わせ（整列）の技術です。

> Image registration is the task of finding a spatial transformation mapping one image to another.

![registration](figure/registration.png)
elastix manualのFig. 2.1を改変。$I_M(T(x))$が$I_F(x)$に整列する$T(x)$を探索する。

出典：S. Klein, M. Staring. Elastix, the manual, v5.0.1. https://github.com/SuperElastix/elastix/releases/download/5.0.1/elastix-5.0.1-manual.pdf, 2020.  

以下のパラメータの調整が可能です。
- Metrics：類似性評価指標
- Image samplers：Fixed imageのサンプリング
- Interpolators：Moving imageの補完
- Transforms：変換
  - Rigid（剛体変換）
  - Non-rigid（非剛体変換）
      - Affine（アフィン変換）
      - Non-affine（非アフィン変換）
        - B-splines
        - Thin-plate splines
- Optimisers：最適化アルゴリズム
- Multi-resolution：データと変換の複雑性について、大域的で荒いものからより細かい設定に段階的に移行させる。そのステップ数や解像度などを設定する。

今回はデフォルトのパラメータ設定で実行します（ただしDefaultPixelValueを-1000に変更）。  
https://github.com/SuperElastix/ElastixModelZoo/tree/master/models/default

###　ボリュームデータの準備
MetaIO（.mhd）形式で保存します。

In [ ]:
os.makedirs('mhd', exist_ok=True)
images = dict()
for dir_name in os.listdir("./dicom"):
    if "PRI" in dir_name:
        path_to_dir = os.path.join("./dicom", dir_name)
        images[dir_name] = read_dicom(path_to_dir)
        sitk.WriteImage(images[dir_name], "./mhd/" + dir_name + ".mhd")

In [ ]:
fig = plt.figure(figsize=(20,10))
fig.add_subplot(1, 3, 1)
plt.title("PRI-7379 (fixed image)")
imshow(images['PRI-7379'][250], phys_axis=True)
fig.add_subplot(1, 3, 2)
imshow(images['PRI-7379'][250], alpha=1, phys_axis=True)
imshow(images['PRI-9361'][250], alpha=0.3, cmap='jet', phys_axis=True)
fig.add_subplot(1, 3, 3)
plt.title("PRI-9361 (moving image)")
imshow(images['PRI-9361'][250], phys_axis=True)

### 剛体変換（rigid transformation）
形状とサイズの変化を伴わない、回転と平行移動のみの変換。

In [ ]:
os.makedirs("reg/rigid", exist_ok=True)
subprocess.run("elastix -f mhd/PRI-7379.mhd -m mhd/PRI-9361.mhd -out reg/rigid -p parameters/Parameters_Rigid.txt", shell=True)
# subprocess.run("source ~/.zshrc; elastix -f mhd/PRI-7379.mhd -m mhd/PRI-9361.mhd -out reg/rigid -p parameters/Parameters_Rigid.txt", shell=True)

In [ ]:
reg_rigid = sitk.ReadImage("./reg/rigid/result.0.mhd")

In [ ]:
fig = plt.figure(figsize=(20,10))
fig.add_subplot(1, 3, 1)
plt.title("PRI-7379 (fixed image)")
imshow(images['PRI-7379'][250], phys_axis=True)
fig.add_subplot(1, 3, 2)
imshow(images['PRI-7379'][250], alpha=1, phys_axis=True)
imshow(reg_rigid[250], alpha=0.3, cmap='jet', phys_axis=True)
fig.add_subplot(1, 3, 3)
plt.title("deformed PRI-9361 (moving image)")
imshow(reg_rigid[250], phys_axis=True)

### アフィン変換（affine transformation）

拡大縮小とせん断（スキュー）を含む変換。

In [ ]:
os.makedirs("reg/affine", exist_ok=True)
subprocess.run("elastix -f mhd/PRI-7379.mhd -m mhd/PRI-9361.mhd -out reg/affine -p parameters/Parameters_Rigid.txt -p parameters/Parameters_Affine.txt", shell=True)
# subprocess.run("source ~/.zshrc; elastix -f mhd/PRI-7379.mhd -m mhd/PRI-9361.mhd -out reg/affine -p parameters/Parameters_Rigid.txt -p parameters/Parameters_Affine.txt", shell=True)

In [ ]:
reg_affine = sitk.ReadImage("./reg/affine/result.1.mhd")

In [ ]:
fig = plt.figure(figsize=(20,10))
fig.add_subplot(1, 3, 1)
plt.title("PRI-7379 (fixed image)")
imshow(images['PRI-7379'][250], phys_axis=True)
fig.add_subplot(1, 3, 2)
imshow(images['PRI-7379'][250], alpha=1, phys_axis=True)
imshow(reg_affine[250], alpha=0.3, cmap='jet', phys_axis=True)
fig.add_subplot(1, 3, 3)
plt.title("deformed PRI-9361 (moving image)")
imshow(reg_affine[250], phys_axis=True)

### 非アフィン変換
屈曲を含む変換。今回はB-splinesを利用します。  
注：計算に数分間かかる可能性があリます。

In [ ]:
os.makedirs("reg/bspline", exist_ok=True)
subprocess.run("elastix -f mhd/PRI-7379.mhd -m mhd/PRI-9361.mhd -out reg/bspline -p parameters/Parameters_Rigid.txt -p parameters/Parameters_Affine.txt -p parameters/Parameters_BSpline.txt", shell=True)
# subprocess.run("source ~/.zshrc; elastix -f mhd/PRI-7379.mhd -m mhd/PRI-9361.mhd -out reg/bspline -p parameters/Parameters_Rigid.txt -p parameters/Parameters_Affine.txt -p parameters/Parameters_BSpline.txt", shell=True)

In [ ]:
reg_bspline = sitk.ReadImage("./reg/bspline/result.2.mhd")

In [ ]:
fig = plt.figure(figsize=(20,10))
fig.add_subplot(1, 3, 1)
plt.title("PRI-7379 (fixed image)")
imshow(images['PRI-7379'][250], phys_axis=True)
fig.add_subplot(1, 3, 2)
imshow(images['PRI-7379'][250], alpha=1, phys_axis=True)
imshow(reg_bspline[250], alpha=0.3, cmap='jet', phys_axis=True)
fig.add_subplot(1, 3, 3)
plt.title("deformed PRI-9361 (moving image)")
imshow(reg_bspline[250], phys_axis=True)

B-splines変換のmulti-resolutionのステップ数と解像度を調整し再度実行します。  

- NumberOfResolutions 4 → 6
- FinalGridSpacingInPhysicalUnits 16 → 8 

注：計算に数分〜数十分間かかる可能性があります。

In [ ]:
os.makedirs("reg/bspline2", exist_ok=True)
subprocess.run("elastix -f mhd/PRI-7379.mhd -m mhd/PRI-9361.mhd -out reg/bspline2 -p parameters/Parameters_Rigid.txt -p parameters/Parameters_Affine.txt -p parameters/Parameters_BSpline2.txt", shell=True)
# subprocess.run("source ~/.zshrc; elastix -f mhd/PRI-7379.mhd -m mhd/PRI-9361.mhd -out reg/bspline2 -p parameters/Parameters_Rigid.txt -p parameters/Parameters_Affine.txt -p parameters/Parameters_BSpline2.txt", shell=True)

In [ ]:
reg_bspline2 = sitk.ReadImage("./reg/bspline2/result.2.mhd")

In [ ]:
fig = plt.figure(figsize=(20,10))
fig.add_subplot(1, 3, 1)
plt.title("PRI-7379 (fixed image)")
imshow(images['PRI-7379'][250], phys_axis=True)
fig.add_subplot(1, 3, 2)
imshow(images['PRI-7379'][250], alpha=1, phys_axis=True)
imshow(reg_bspline2[250], alpha=0.3, cmap='jet', phys_axis=True)
fig.add_subplot(1, 3, 3)
plt.title("deformed PRI-9361 (moving image)")
imshow(reg_bspline2[250], phys_axis=True)

余力があれば以下も確認してみてください。
- 各種パラメータを調整して変化を確認。  
- 別種（PRI-3027）とのレジストレーションも可能か。

### レジストレーションの応用

- 自動ランドマーキング
- セグメンテーション
- 平均ボリュームデータの算出  
など

今回は、先ほどelastixで算出したトランスフォームパラメータを利用して、自動ランドマーキングを試みます。  
1. 3D Slicerを利用して、fixed imageのランドマークを手動で取得する（[参考](https://itots.github.io/morphometrics_lecture/index.html#34_%E3%83%A9%E3%83%B3%E3%83%89%E3%83%9E%E3%83%BC%E3%82%AF%E3%83%87%E3%83%BC%E3%82%BF%E3%81%AE%E5%8F%96%E5%BE%97)) 。
2. これをトランスフォームすることで、moving imageにマッピングする。

3D Slicerのfscvファイルをelastixのtransformixが読み込める形式に変換します。

In [ ]:
def fcsv2trx(fcsv_file, trx_file):
    df = pd.read_csv(fcsv_file, skiprows=2)
    df = df.iloc[:,1:4]
    with open(trx_file, 'w') as f:
        f.write("point\n")
        f.write(str(len(df))+"\n")
        for index, row in df.iterrows():
            f.write(str(row[0]) + '\t' + str(row[1]) + '\t' + str(row[2]) + '\n')

In [ ]:
os.makedirs('transformix/fixed', exist_ok=True)
fcsv2trx(fcsv_file='landmarks/PRI-7379.fcsv', trx_file='transformix/fixed/PRI-7379.txt')

elastixの`transformix`を利用してfixed imageのランドマークを変換し、moving imageにマッピングします。

In [ ]:
os.makedirs('transformix/moving/PRI-9361', exist_ok=True)
subprocess.run("transformix -def transformix/fixed/PRI-7379.txt -out transformix/moving/PRI-9361 -tp reg/bspline2/TransformParameters.2.txt", shell=True)
# subprocess.run("source ~/.zshrc; transformix -def transformix/fixed/PRI-7379.txt -out transformix/moving/PRI-9361 -tp reg/bspline2/TransformParameters.2.txt", shell=True)

`transformix`の出力結果をfcsv形式に変換します。

In [ ]:
def trx2fcsv(trx_file, fcsv_file):
    with open(trx_file) as f:
        points = []
        for line in f:
            points += [line.split(";")[4].split(" ")[4:7]]
    df = pd.DataFrame(np.array(points, dtype = float), columns=list('xyz'))
    with open(fcsv_file, 'w') as f:
        f.write('# Markups fiducial file version = 4.11\n')
        f.write('# CoordinateSystem = LPS\n')
        f.write('# columns = id,x,y,z\n')
        for index, row in df.iterrows():
            f.write(str(index+1)+','+str(row[0])+','+str(row[1])+','+str(row[2])+'\n')

In [ ]:
os.makedirs('transformix/fcsv', exist_ok=True)
trx2fcsv(trx_file='transformix/moving/PRI-9361/outputpoints.txt', fcsv_file='transformix/fcsv/PRI-9361.fcsv')

3D Slicerに読み込んで、手動計測の結果と比較してみてください。  

なお、レジストレーションに基づく自動ランドマーキングは、深層学習と組み合わせることで精度が向上することが報告されています（Devine et al. 2020）。  
また、今回お示ししたような画像のレジストレーションではなく、点群のレジストレーションを利用した自動ランドマーキングの方法も開発されています（Porto et al. 2021）。3D Slicerのモジュール（[Slicer Morph](https://slicermorph.github.io/)）として利用できるようです。

## 参考・引用文献

- SimpleITK Sphinx Documentation. https://simpleitk.readthedocs.io/en/master/index.html
- SimpleITK Notebooks. https://github.com/InsightSoftwareConsortium/SimpleITK-Notebooks
- S. Klein, M. Staring. Elastix, the manual, v5.0.1. https://github.com/SuperElastix/elastix/releases/download/5.0.1/elastix-5.0.1-manual.pdf, 2020.
- R. Beare, B. C. Lowekamp, Z. Yaniv, “Image Segmentation, Registration and Characterization in R with SimpleITK”, J Stat Softw, 86(8), https://doi.org/10.18637/jss.v086.i08, 2018.
- Z. Yaniv, B. C. Lowekamp, H. J. Johnson, R. Beare, “SimpleITK Image-Analysis Notebooks: a Collaborative Environment for Education and Reproducible Research”, J Digit Imaging., https://doi.org/10.1007/s10278-017-0037-8, 31(3): 290-303, 2018.
- B. C. Lowekamp, D. T. Chen, L. Ibáñez, D. Blezek, “The Design of SimpleITK”, Front. Neuroinform., 7:45. https://doi.org/10.3389/fninf.2013.00045, 2013.
- S. Klein, M. Staring, K. Murphy, M.A. Viergever and J.P.W. Pluim, "elastix: a toolbox for intensity based medical image registration," IEEE Transactions on Medical Imaging, vol. 29, no. 1, pp. 196 - 205, January 2010. download doi
- D.P. Shamonin, E.E. Bron, B.P.F. Lelieveldt, M. Smits, S. Klein and M. Staring, "Fast Parallel Image Registration on CPU and GPU for Diagnostic Classification of Alzheimer's Disease", Frontiers in Neuroinformatics, vol. 7, no. 50, pp. 1-15, January 2014. download doi
- Kikinis R, Pieper SD, Vosburgh K (2014) 3D Slicer: a platform for subject-specific image analysis, visualization, and clinical support. Intraoperative Imaging Image-Guided Therapy, Ferenc A. Jolesz, Editor 3(19):277–289 ISBN: 978-1-4614-7656-6 (Print) 978-1-4614-7657-3 (Online)
- Sobel operator: https://en.wikipedia.org/wiki/Sobel_operator
- Devine, Jay, Jose D. Aponte, David C. Katz, Wei Liu, Lucas D. Lo Vercio, Nils D. Forkert, Ralph Marcucio, Christopher J. Percival, and Benedikt Hallgrímsson. 2020. “A Registration and Deep Learning Approach to Automated Landmark Detection for Geometric Morphometrics.” Evolutionary Biology 47 (3): 246–59.
- Porto, Arthur, Sara Rolfe, and A. Murat Maga. 2021. “ALPACA: A Fast and Accurate Computer Vision Approach for Automated Landmarking of Three‐dimensional Biological Structures.” Methods in Ecology and Evolution / British Ecological Society, no. 2041-210X.13689 (August). https://doi.org/10.1111/2041-210x.13689.